# Types: a fundamental feature

Although we can go a long way with C or Matlab style coding (and have it be efficient), the power of Julia is bound up with how its **type system** works. 

A type is a label that tells Julia how to interpret a given collection of data.



# Multiple dispatch

## Example: Diagonal matrices 

- In mathematics, its common to use matrices with certain special properties, e.g. Diagonal, SymTriDiag


- This is reflected in Julia by *using different types*


- A type tells Julia how to "behave", via multiple dispatch

In [2]:
D = Diagonal([1.0, 2, 3])

3×3 Diagonal{Float64}:
 1.0   ⋅    ⋅ 
  ⋅   2.0   ⋅ 
  ⋅    ⋅   3.0

We see that there is a special `show` method defined for objects of `Diagonal` type, that is automatically called.

We can *introspect*, i.e. ask Julia for more information about the type:

In [7]:
fieldnames(Diagonal)

1-element Array{Symbol,1}:
 :diag

In [8]:
D.diag

3-element Array{Float64,1}:
 1.0
 2.0
 3.0

We see that *only* the diagonal elements are actually stored inside the object: the zeros in the rest of the matrix are *not* explicitly stored (i.e. it is a particular kind of sparse matrix).

What happens if we multiply a diagonal matrix by a vector?

In [11]:
v = [4.0, 5, 6]
D * v

3-element Array{Float64,1}:
  4.0
 10.0
 18.0

In [12]:
@which D * v

*(D::Diagonal, V::AbstractArray{T,1} where T) in Base.LinAlg at linalg/diagonal.jl:147

Julia tells us precisely which method is called, which does

In [14]:
D.diag .* v

3-element Array{Float64,1}:
  4.0
 10.0
 18.0

That is, a pointwise product element by element of the two vectors - exactly what the mathematics tells us it should do.
The method (version of the function `*`) is chosen by **dispatching** (choosing based on) the types of all of the arguments (**multiple**). This enables the most efficient version to be used when possible, or else a generic version to be used instead.

An even more striking example is that of finding eigenvalues of a symmetric tridiagonal matrix, which again uses a specialised method.

# Defining types and operations

We as users can define our own types; these can be thought of as templates that determine how objects are made.

## Example: Automatic differentiation 

A powerful example is to define a type that represents the value of a function $f$ at a point $a$, together with the value of its derivative there, $f'(a)$. By defining arithmetic operations and functions on these objects in a suitable way, we can perform **automatic (or algorithmic) differentiation**

We represent a function $f$ by the pair $(f(a), f'(a))$, which is often called a "dual number".

In [5]:
struct FloatDual
    value::Float64
    deriv::Float64
end

The operations on `Jet`s should correspond to the operations on those functions to give new functions. For example, we have

In [6]:
import Base: +, *   # import functions to be able to extend / overload them

+(f::FloatDual, g::FloatDual) = FloatDual(f.value+g.value, f.deriv+g.deriv)
*(f::FloatDual, g::FloatDual) = FloatDual(f.value*g.value, f.value*g.deriv + f.deriv*g.value)

* (generic function with 185 methods)

We also want to be able to multiply be a scalar (real number):

In [7]:
*(α::Real, f::FloatDual) = FloatDual(α*f.value, α*f.deriv)

* (generic function with 186 methods)

A constant function $c(x) := c$  for all $x$ has derivative $0$, so `c` corresponds to the dual number `FloatDual(c, 0)`.
The identity function $\mathrm{id}(x)$ has the value $\mathrm{id}(a)=a$ at the point $x=a$ and derivative $1$, and so corresponds to the dual number `FloatDual(a, 1)`.

We can now do simple derivatives. Let's calculate the derivative of the function

In [8]:
f(x) = x^2 + (-2)*x   # we haven't defined - for `FloatDual`; ^ comes for free

f (generic function with 1 method)

at the point

In [13]:
a = 3.0

3.0

We define an object

In [14]:
xx = FloatDual(a, 1)

FloatDual(3.0, 1.0)

and evaluate `f` on it:

In [15]:
f(xx)

FloatDual(3.0, 4.0)

The first entry is indeed the value of 

In [16]:
f(a)

3.0

and the second is the value of the derivative of the function $f$ at the point $x=a$, calulcated automatically.

Note that we used the following "generic fallback" method, which is called when no more specific method is available:

In [38]:
@which xx^2

^(x, p::Integer) in Base at intfuncs.jl:196

Note that several [implementations](http://www.juliadiff.org/) of automatic differentiation are available; we should mention, in particular, `ForwardDiff.jl` and `ReverseDiff.jl`.

# Generic programming and composability

The above type can only be used to differentiate functions of `Float64`, since that's the type we imposed in the definition of `FloatDual`. If we wanted to differentiate a different type, we would have to make a new type. 

Julia instead allows us to create a whole *parameterized* family of types with the same structure, but containing objects with different types:

In [1]:
struct Dual{T<:Real}
    value::T
    deriv::T
end

Here we have specified that we want to allow any type `T` that is a *subtype* (`<:`) of `Real`. For example:

In [2]:
Dual(3, 4)

Dual{Int64}(3, 4)

The type is automatically "discovered" (inferred).

In [3]:
Dual(3.0, 4.0)

Dual{Float64}(3.0, 4.0)

Repeating the previous definitions, we have

In [4]:
import Base: +, *

In [5]:
+(f::Dual{T}, g::Dual{T}) where {T<:Real} = Dual(f.value+g.value, f.deriv+g.deriv)
*(f::Dual{T}, g::Dual{T}) where {T<:Real} = Dual(f.value*g.value, f.value*g.deriv + f.deriv*g.value)

*(α::Real, f::Dual{T}) where {T<:Real} = Dual(α*f.value, α*f.deriv)

* (generic function with 186 methods)

Now we can define a function to differentiate a function `f` as 

In [6]:
differentiate(f, a) = f(Dual(a, one(a))).deriv 

differentiate (generic function with 1 method)

In [7]:
g(x) = x^2 + (-2)x

g (generic function with 1 method)

In [8]:
differentiate(g, 3)

4

In [9]:
differentiate(g, 3.0)

4.0

What is more interesting, however, is that we can now use *any type whatsoever* inside. This allows a very high degree of **composability**.

For example, the package `IntervalArithmetic.jl` defines arithmetic operations on intervals, with the property that the result of a calculation with intervals contains the result of the calculation with all possible inputs.

In [10]:
using IntervalArithmetic

In [11]:
differentiate(g, Interval(3, 4))

[4, 6]

The result is an interval that is *guaranteed to contain the value of the derivative over the whole input interval*.

## Code specialization

It is important to emphasise that for each different type, Julia is compiling a *specialized version of the code*. This leads to excellent performance.

# New syntax and possibilities

How could we take the `sin` of a `Dual`?

In [12]:
Base.sin(f::Dual{T}) where {T<:Real} = Dual(sin(f.value), cos(f.value)*f.deriv)

The `where` clause means that this applies to a *union* of types, i.e. the collection of all types `Dual{T}` with the given restriction on `T`.

If there is no restriction, this can be written `T<:Any`, or just `where {T}`.

There is a shorthand syntax for this in the case that we do not need to use the explicit value of `T` in the function definition:

In [13]:
Base.sin(f::Dual{<:Real}) = Dual(sin(f.value), cos(f.value)*f.deriv)

## "Triangular dispatch"

Suppose we want to define a type that needs access to a type `T` and to a `Vector` of that type. We can write this as follows:

In [2]:
struct HasData{T, S<:AbstractVector{T}} 
    data::S
end

In [3]:
1:3 isa AbstractVector

true

However, we seem not to be able to create an object of our type:

In [4]:
HasData(1:3)

LoadError: [91mMethodError: Cannot `convert` an object of type UnitRange{Int64} to an object of type HasData
This may have arisen from a call to the constructor HasData(...),
since type constructors fall back to convert methods.[39m

In fact we can:

In [5]:
HasData{Int64, UnitRange{Int64}}(1:3)

HasData{Int64,UnitRange{Int64}}(1:3)

Of course, we want Julia to work that all out for us, so we add an **outer constructor**:

In [6]:
HasData(data::AbstractVector{T}) where {T} = HasData{T, typeof(data)}(data)  # define an outer constructor

HasData

In [7]:
d = HasData(1:3)

HasData{Int64,UnitRange{Int64}}(1:3)

In [8]:
HasData([1,2,3])

HasData{Int64,Array{Int64,1}}([1, 2, 3])

# Functions in types

Often, we want to have a function inside a type. The obvious (but usually wrong) thing to write is

In [1]:
struct MyType
    f::Function
end

In [2]:
using BenchmarkTools

In [6]:
s = MyType(sin)

@btime $s.f(10)

  43.260 ns (1 allocation: 16 bytes)


-0.5440211108893698

In [16]:
@btime sin(10)

  13.864 ns (0 allocations: 0 bytes)


-0.5440211108893698

There is some overhead here.

Instead, we make the *type of the function* a parameter:

In [4]:
struct MyType2{F}
    f::F
end

In [7]:
s2 = MyType2(sin)
@btime $s2.f(10)

  13.855 ns (0 allocations: 0 bytes)


-0.5440211108893698

There is no longer any overhead.

Of course, we don't want to write `s2.f` to run the function. We can instead treat the object as a function object by overloading function calling on it:

In [8]:
(a::MyType2)(x) = a.f(x)

In [10]:
s2(10)

-0.5440211108893698

In [11]:
@btime $s2(10)

  14.294 ns (0 allocations: 0 bytes)


-0.5440211108893698